In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from frvrs_utils import FRVRSUtilities
from notebook_utils import NotebookUtilities
import os.path as osp
import pandas as pd

nu = NotebookUtilities(
    data_folder_path=osp.abspath('../data'),
    saves_folder_path=osp.abspath('../saves')
)
fu = FRVRSUtilities(
    data_folder_path=osp.abspath('../data'),
    saves_folder_path=osp.abspath('../saves')
)


# Rasch Analysis

In [3]:

frvrs_logs_df = nu.load_data_frames(frvrs_logs_df='frvrs_logs_df')['frvrs_logs_df']
print(frvrs_logs_df.shape)

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl.
Argument 'placement' has incorrect type (expected pandas._libs.internals.BlockPlacement, got slice)
No pickle exists for frvrs_logs_df - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv.
(829116, 114)



## Calculate item logits
<p>In Rasch analysis, item logits, also known as item difficulty, represent the location of an item on the latent variable measured by the instrument. They are calculated based on the observed responses of participants to the item. Here's how to compute item logits for a task with 0-1 scores:</p>
<p><strong>1. Calculate the probability of a correct response:</strong></p>
<ul>
    <li>For each participant, determine their score on the task (0 or 1).</li>
    <li>Count the total number of correct responses across all participants.</li>
    <li>Divide the number of correct responses by the total number of participants. This gives you the probability (P) of a correct response for the task.</li>
</ul>
<p><strong>2. Convert the probability to log-odds (logit):</strong></p>
<ul>
    <li>Use the natural logarithm (ln) function to calculate the log-odds of a correct response:</li>
</ul>
<quote>
    <pre><code>Logit = ln(P/(1-P))</code></pre>
</quote>
<p><strong>3. Adjust for item calibration:</strong></p>
<ul>
    <li>In Rasch analysis, item logits are typically estimated iteratively within a calibration process. This process considers the responses to all items and participants to adjust the initial item logit estimates.</li>
    <li>Software programs such as Winsteps, RUMM2030, and Facets can be used to perform Rasch analysis and provide calibrated item logit estimates.</li>
</ul>
<p><strong>Additional Considerations:</strong></p>
<ul>
    <li>Item difficulty is interpreted relative to the ability of the participants. A positive logit value indicates that the item is difficult, requiring a higher ability level to answer correctly. Conversely, a negative logit value indicates that the item is easier, requiring a lower ability level to answer correctly.</li>
    <li>The precision of the item logit estimate depends on several factors, including the sample size and the distribution of participant abilities. Larger sample sizes and a wider range of abilities will generally lead to more precise estimates.</li>
</ul>

In [4]:

import numpy as np

# For each score, determine their scores on each task (0 or 1)
base_mask_series = (frvrs_logs_df.scene_type == 'Triage') & (frvrs_logs_df.is_scene_aborted == False) & (frvrs_logs_df.is_a_one_triage_file == True)
gb = frvrs_logs_df[base_mask_series].sort_values(['action_tick']).groupby(fu.scene_groupby_columns)
rows_list = []
for (session_uuid, scene_id), scene_df in gb:
    is_stills_visited_first = fu.get_stills_value(scene_df)
    is_walk_command_issued = fu.get_walk_value(scene_df)
    is_walkers_visited_last = fu.get_walkers_value(scene_df)
    is_wave_command_issued = fu.get_wave_value(scene_df)
    for patient_id, patient_df in scene_df.groupby('patient_id'):
        is_pulse_taken = fu.get_pulse_value(patient_df)
        is_tag_correct = fu.get_tag_value(patient_df)
        for injury_id, injury_df in patient_df.groupby('injury_id'):
            row_dict = {}
            for cn in fu.patient_groupby_columns: row_dict[cn] = eval(cn)
            row_dict['is_stills_visited_first'] = is_stills_visited_first
            row_dict['is_walk_command_issued'] = is_walk_command_issued
            row_dict['is_walkers_visited_last'] = is_walkers_visited_last
            row_dict['is_wave_command_issued'] = is_wave_command_issued
            row_dict['is_injury_treated'] = fu.get_treatment_value(patient_df, injury_id)
            row_dict['is_pulse_taken'] = is_pulse_taken
            row_dict['is_tag_correct'] = is_tag_correct
            rows_list.append(row_dict)
item_logits_df = pd.DataFrame(rows_list)

In [5]:

print(item_logits_df.is_tag_correct.sum())
assert item_logits_df.applymap(lambda x: str(x) in ['None', 'nan']).sum().sum() == 0, "You have nulls in your data"

5118


In [6]:

# Count the total number of correct responses across all scenes
srs = item_logits_df.sum()
correct_stills_count = srs.is_stills_visited_first
correct_walk_count = srs.is_walk_command_issued
correct_walkers_count = srs.is_walkers_visited_last
correct_wave_count = srs.is_wave_command_issued
correct_injury_count = srs.is_injury_treated
correct_pulse_count = srs.is_pulse_taken
correct_tag_count = srs.is_tag_correct

In [7]:

# Divide the number of correct responses by the total number of scenes
# This gives you the probability (P) of a correct response for each task
scene_count = item_logits_df.shape[0]
correct_stills_probability = correct_stills_count / scene_count
correct_walk_probability = correct_walk_count / scene_count
correct_walkers_probability = correct_walkers_count / scene_count
correct_wave_probability = correct_wave_count / scene_count
correct_injury_probability = correct_injury_count / scene_count
correct_pulse_probability = correct_pulse_count / scene_count
correct_tag_probability = correct_tag_count / scene_count

In [8]:

# Use the natural logarithm (ln) function to calculate the log-odds of a correct response
import math

try: correct_stills_logodds = math.log(correct_stills_probability / (1 - correct_stills_probability))
except: correct_stills_logodds = np.nan
try: correct_walk_logodds = math.log(correct_walk_probability / (1 - correct_walk_probability))
except: correct_walk_logodds = np.nan
try: correct_walkers_logodds = math.log(correct_walkers_probability / (1 - correct_walkers_probability))
except: correct_walkers_logodds = np.nan
try: correct_wave_logodds = math.log(correct_wave_probability / (1 - correct_wave_probability))
except: correct_wave_logodds = np.nan
try: correct_injury_logodds = math.log(correct_injury_probability / (1 - correct_injury_probability))
except: correct_injury_logodds = np.nan
try: correct_pulse_logodds = math.log(correct_pulse_probability / (1 - correct_pulse_probability))
except: correct_pulse_logodds = np.nan
try: correct_tag_logodds = math.log(correct_tag_probability / (1 - correct_tag_probability))
except: correct_tag_logodds = np.nan

In [9]:

print(f'''
Log-odds of correctly visiting still patients (first): {correct_stills_logodds}
Log-odds of issuing the "walk" command: {correct_walk_logodds}
Log-odds of correctly visiting walking patients (last): {correct_walkers_logodds}
Log-odds of issuing the "wave" command: {correct_wave_logodds}
Log-odds of correctly treating an injury: {correct_injury_logodds}
Log-odds of taking a pulse: {correct_pulse_logodds}
Log-odds of correctly tagging a patient: {correct_tag_logodds}''')


Log-odds of correctly visiting still patients (first): -1.307552889804326
Log-odds of issuing the "walk" command: 3.385449024694017
Log-odds of correctly visiting walking patients (last): -0.4027851211222216
Log-odds of issuing the "wave" command: 1.4869252322225146
Log-odds of correctly treating an injury: -0.3140038792048232
Log-odds of taking a pulse: 0.86385487698838
Log-odds of correctly tagging a patient: 1.089858220508195


In [14]:

# Define convergence threshold
threshold = 0.001

# Initialize item logits and scores
item_logits = np.array([
    correct_stills_logodds, correct_walk_logodds, correct_walkers_logodds, correct_wave_logodds, correct_injury_logodds, correct_pulse_logodds, correct_tag_logodds
])
columns_list = [
    'is_stills_visited_first', 'is_walk_command_issued', 'is_walkers_visited_last', 'is_wave_command_issued', 'is_injury_treated', 'is_pulse_taken', 'is_tag_correct'
]
assert len(item_logits) == len(columns_list), 'The item logits need to be the same count as the number of columns in scores'
df = item_logits_df[columns_list]
scores = df.values
assert df.applymap(lambda x: x not in [0, 1]).sum().sum() == 0, 'You have non-Bernoulli data'

# Define the number of iterations
iterations = 100

print(
    f'In Rasch analysis, if the shape of item_logits was {item_logits.shape} and the shape of scores was {scores.shape},'
    ' what would we expect the shape of scene_estimates to be in this iterative calibration process?'
)

In Rasch analysis, if the shape of item_logits was (7,) and the shape of scores was (6839, 7), what would we expect the shape of scene_estimates to be in this iterative calibration process?


In [23]:

def analyze_data(item_logits, scores, verbose=False):
    """
    Estimate scene characteristics using an iterative proportional fitting (IPF)
    algorithm for the Rasch model.

    Parameters:
        item_logits (numpy.ndarray): An array of item logits representing the
                                     log-odds of scoring on the item.
        scores (numpy.ndarray): A binary matrix where each row represents an injury-
                                patient-scene and each column an item, indicating
                                the pass-fail score the injury got on the test item.
        verbose (bool, optional): If True, print intermediate results for debugging.
                                  Default is False.

    Returns:
        scene_estimates (numpy.ndarray): An array containing scene estimates based
                                         on the Rasch model.
        updated_item_logits (numpy.ndarray): Updated item logits after the
                                             iteration.

    Note:
        The Rasch model is typically estimated using maximum likelihood estimation
        (MLE) methods, which involve iteratively updating the item parameters and
        the person abilities until convergence is achieved. The algorithm used
        here is a type of iterative proportional fitting (IPF) algorithm, which is
        a type of MLE algorithm that is commonly used to estimate IRT models like
        the Rasch model. The IPF algorithm is similar to the EM algorithm in that
        it involves iteratively updating the item parameters and the person
        abilities until convergence is achieved. However, the IPF algorithm is
        simpler than the EM algorithm because it does not require the calculation
        of expected sufficient statistics.
    """
    if verbose: print('\nitem_logits', item_logits.shape, '\n', item_logits)
    if verbose: print('\nscores', scores.shape, '\n', scores)
    
    # Calculate the expected scores for each item
    expected_scores = 1 / (1 + np.exp(-item_logits))
    if verbose: print('\nexpected_scores', expected_scores.shape, '\n', expected_scores)
    
    # Calculate the observed scores for each item
    observed_scores = scores.mean(axis=0)
    if verbose: print('\nobserved_scores', observed_scores.shape, '\n', observed_scores)
    
    # Calculate the difference between the observed and expected scores for each item
    score_diffs = observed_scores - expected_scores
    if verbose: print('\nscore_diffs', score_diffs.shape, '\n', score_diffs)
    
    # Calculate the sum of the score differences for each item
    score_sums = score_diffs.sum()
    if verbose: print('\nscore_sums', score_sums.shape, '\n', score_sums)
    
    # Update the item logits for the next iteration
    updated_item_logits = item_logits + score_sums
    if verbose: print('\nupdated_item_logits', updated_item_logits.shape, '\n', updated_item_logits)
    
    # Calculate the scene estimates using the updated item logits
    scene_estimates = np.sum(scores - 0.5, axis=1) - np.sum((expected_scores - 0.5).reshape(1, -1), axis=1)
    if verbose: print('\nscene_estimates', scene_estimates.shape, '\n', scene_estimates)
    
    return scene_estimates, updated_item_logits

In [24]:

verbose = True
for _ in range(iterations):
    
    # Perform Rasch analysis using the current item logits
    scene_estimates, updated_item_logits = analyze_data(item_logits, scores, verbose=verbose)
    verbose = False
    assert scene_estimates.shape == (scores.shape[0],), 'The estimated ability of each scene has the wrong shape'
    assert updated_item_logits.shape == item_logits.shape, 'The shape of the new difficulty levels for each item after the current iteration is wrong'
    
    # Check for convergence
    if max(abs(updated_item_logits - item_logits)) <= threshold: break
    
    # Update item logits for the next iteration
    item_logits = updated_item_logits


item_logits (7,) 
 [-1.30755289  3.38544902 -0.40278512  1.48692523 -0.31400388  0.86385488
  1.08985822]

scores (6839, 7) 
 [[0 1 0 ... 0 1 0]
 [0 1 0 ... 0 1 0]
 [0 1 0 ... 0 1 0]
 ...
 [0 1 0 ... 1 0 1]
 [0 1 0 ... 1 0 0]
 [0 1 0 ... 1 0 0]]

expected_scores (7,) 
 [0.21289662 0.96724667 0.40064337 0.81561632 0.42213774 0.70346542
 0.74835502]

observed_scores (7,) 
 [0.21289662 0.96724667 0.40064337 0.81561632 0.42213774 0.70346542
 0.74835502]

score_diffs (7,) 
 [ 2.77555756e-17  0.00000000e+00  0.00000000e+00 -1.11022302e-16
 -5.55111512e-17  0.00000000e+00  0.00000000e+00]

score_sums () 
 -1.3877787807814457e-16

updated_item_logits (7,) 
 [-1.30755289  3.38544902 -0.40278512  1.48692523 -0.31400388  0.86385488
  1.08985822]

scene_estimates (6839,) 
 [-1.27036116 -1.27036116 -1.27036116 ... -0.27036116 -1.27036116
 -1.27036116]


In [26]:

print(f'''
Log-odds of correctly visiting still patients (first): {updated_item_logits[0]}
Log-odds of issuing the "walk" command: {updated_item_logits[1]}
Log-odds of correctly visiting walking patients (last): {updated_item_logits[2]}
Log-odds of issuing the "wave" command: {updated_item_logits[3]}
Log-odds of correctly treating an injury: {updated_item_logits[4]}
Log-odds of taking a pulse: {updated_item_logits[5]}
Log-odds of correctly tagging a patient: {updated_item_logits[6]}''')


Log-odds of correctly visiting still patients (first): -1.3075528898043263
Log-odds of issuing the "walk" command: 3.385449024694017
Log-odds of correctly visiting walking patients (last): -0.40278512112222176
Log-odds of issuing the "wave" command: 1.4869252322225144
Log-odds of correctly treating an injury: -0.3140038792048233
Log-odds of taking a pulse: 0.8638548769883799
Log-odds of correctly tagging a patient: 1.0898582205081948


In [15]:

# Calculate the Fisher information
correct_stills_fisher = correct_stills_probability * (1 - correct_stills_probability)
correct_walk_fisher = correct_walk_probability * (1 - correct_walk_probability)
correct_walkers_fisher = correct_walkers_probability * (1 - correct_walkers_probability)
correct_wave_fisher = correct_wave_probability * (1 - correct_wave_probability)
correct_injury_fisher = correct_injury_probability * (1 - correct_injury_probability)
correct_pulse_fisher = correct_pulse_probability * (1 - correct_pulse_probability)
correct_tag_fisher = correct_tag_probability * (1 - correct_tag_probability)

In [16]:

# Calculate the model standard error for item measures
correct_stills_mse = 1 / math.sqrt(correct_stills_fisher)
correct_walk_mse = 1 / math.sqrt(correct_walk_fisher)
correct_walkers_mse = 1 / math.sqrt(correct_walkers_fisher)
correct_wave_mse = 1 / math.sqrt(correct_wave_fisher)
correct_injury_mse = 1 / math.sqrt(correct_injury_fisher)
correct_pulse_mse = 1 / math.sqrt(correct_pulse_fisher)
correct_tag_mse = 1 / math.sqrt(correct_tag_fisher)

In [17]:

print(f'''
correct_stills_mse = {correct_stills_mse}
correct_walk_mse = {correct_walk_mse}
correct_walkers_mse = {correct_walkers_mse}
correct_wave_mse = {correct_wave_mse}
correct_injury_mse = {correct_injury_mse}
correct_pulse_mse = {correct_pulse_mse}
correct_tag_mse = {correct_tag_mse}''')


correct_stills_mse = 2.241692719274822
correct_walk_mse = 5.618283762312336
correct_walkers_mse = 2.015443147981806
correct_wave_mse = 2.5786702240404464
correct_injury_mse = 2.024700284262668
correct_pulse_mse = 2.1894798361849324
correct_tag_mse = 2.304369019293232
